# 클러스터링 (Clustering) 실습

In [1]:
#구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# 입력 데이터 위치
Path = '/content/drive/MyDrive/Colab Notebooks/KT_RNN/data/clustering/' 

# 각 주제별 100건의 뉴스 데이터
# Read an Excel file into a pandas DataFrame
# DataFrame : 2차원 데이터구조. Row, Column, Series(각 Column에 있는 데이터들)로 구성
climate = pd.read_excel(Path+'news_climate change.xlsx')
mobility = pd.read_excel(Path+'news_mobility.xlsx')
probiotics = pd.read_excel(Path+'news_probiotics.xlsx')

# 주제 column을 추가
climate['주제'] = '기후위기'
mobility['주제'] = '모빌리티'
probiotics['주제'] = '유산균'

# 각 주제별 70건의 뉴스데이터를 가져와서 입력용 데이터를 생성
# pandas.concat(objs, axis=0, ignore_index=Faslse, ...)
# axis=0, 0: 위+아래로 합치기, 1: 왼쪽+오른쪽으로 합치기
# ignore_index=True : 기존 index를 무시. The resulting axis will be labeled 0, …, n - 1. 
data = pd.concat([climate[:70], mobility[:70], probiotics[:70]], ignore_index=True)

# 뉴스기사 body, title을 기준으로 중복을 제거
# DataFrame.drop_duplicates : Returns DataFrame with duplicate rows removed.
data_unique = data.drop_duplicates(['body']).drop_duplicates(['title'])

# 중복 제거된 데이터 건수를 확인
# DataFrame.shape : Returns a tuple representing the dimensionality of the DataFrame.
data.shape, data_unique.shape

((210, 5), (198, 5))

중복으로 들어있는 기사를 제거하고 기사들의 순서를 무작위로 섞습니다.


In [4]:
# 데이터 순서를 셔플링
# frac: 랜덤 추출할 비율 (1 == 전체 데이터를 셔플링)
# random_state: 랜덤 추출할 값에 seed 설정하면, 항상 같은 결과를 생성
# reset_index: Reset the index. Use drop parameter to avoid the old index being added as a column
data_shuffled = data_unique.sample(frac=1, random_state=16).reset_index(drop=True) 
data_shuffled.head(3)

,date,title,body,channel,주제
0,2022-08-26 14:36:00,"'자율주행차에서 브레이크 고장나면?'…현대차그룹, 대응 시스템 공개",[자료=현대자동차] 현대차그룹이 오늘(26일) 특허 출원한 자율주행 차량의 브레이크...,news,모빌리티
1,2022-08-16 19:50:00,"미-러 핵전쟁 시 50억명 이상 굶어 죽는다"""" 美 연구팀 발표",미국과 러시아가 핵전쟁을 할 경우 전 세계 50억명 이상의 사람이 굶주림으로 숨질 ...,news,기후위기
2,2022-08-29 20:07:00,"[여기는 전남] 전남도, 적조 피해 예방 총력 대응 외",[KBS 광주]고흥 외나로도에서 여수 돌산 해역까지 적조 주의보가 내려진 가운데 전...,news,기후위기


In [5]:
# DataFrame.iloc : integer-location based indexing for selection by position
data_shuffled.iloc[0]['title']
data_shuffled.iloc[0]['body']

'[자료=현대자동차] 현대차그룹이 오늘(26일) 특허 출원한 자율주행 차량의 브레이크 고장 대응 시스템 을 HMG 테크 사이트 에 공개했습니다. 현대차그룹은 해당 기술이 완전 자율주행 모빌리티를 전제로 하며, 브레이크 장치에 오류가 발생하는 예기치 못한 상황에서 다른 차량 간의 상호작용으로 피해를 최소화하는 사고 대응 기술이라고 설명했습니다. 브레이크 고장 시스템으로, 자율주행 차량이 주행 도중 제동이 필요한 상황에서 전자 제어장치를 통해 메인 제동 장치의 작동을 확인하고, 반응이 없으면 전자 제어장치는 다시 한번 보조 제동 장치를 거쳐 제동 가능 여부를 판별합니다. 이때 보조 제동 장치까지 사용이 불가능하다고 판단되면 전자 제어장치는 가장 가까운 자율주행 관제시스템을 향해 제동 시스템에 문제가 있다는 신호를 전달합니다. 정보를 전달받은 관제시스템은 고장 차량과 가장 가까이에서 주행 중인 차량에게 구조 신호를 보냅니다. 신호를 수신한 자율주행 차량은 고장 차량을 안전하게 정지시키기 위해 고장 차량이 관제시스템에게 전달한 정보를 토대로 조향 시스템을 제어합니다. 또한, 구조 진행 상황을 관제시스템에게 전달함과 동시에 고장 차량과 같은 차선으로 주행하도록 접근합니다. 관제시스템은 고장 차량의 무사 처리를 위해 주변 차량들이 해당 차선에 주행하지 않도록 지시합니다. 같은 차선으로 이동한 차량은 주행 속도와 조향 시스템을 꾸준히 조절하며 고장 차량의 범퍼에 접촉해 차량의 속도를 서서히 줄입니다. 꾸준한 차량 제어로 고장 차량이 완전히 멈추면 관제시스템에 구조 완료 신호를 보낸 후 구조 작업을 끝냅니다. 무선 통신을 거쳐 주변 차량과의 상호작용으로 사고를 완화하는 것입니다. 현대차그룹은 "브레이크 고장 대응 시스템 기술에 대한 특허를 미국, 중국, 한국에 출원했으며, 가능성이 희박한 만약의 상황까지 고려한 전방위 안전 체계를 구축하기 위해 자율주행 기술의 기초부터 완성도를 꾸준히 높여가고 있다"고 말했습니다.'

먼저 형태소 분석을 하기 위해 konlpy를 설치합니다.

In [6]:
# 한국어 형태소 분석기를 포함하고 있는 파이썬 패키지 설치
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 56.5 MB/s 


##**< 뉴스 기사 단어 빈도 분석 >**


오픈소스 한국어 형태소분석기 중 하나인 Hannanum 패키지를 import하고 클래스 객체를 생성합니다.

빈 리스트를 하나 만들고 `hannanum` 클래스를 이용해 명사만을 추출하여 저장합니다.



In [7]:
# 데이터에서 첫번째 기사 내용을 가져와서 명사만 추출합니다
from konlpy.tag import Hannanum
hannanum = Hannanum()

temp = hannanum.nouns(data_shuffled.loc[0]['body'])
print(temp)

['[자료=현대자동차]', '현대차그룹', '오늘(26일)', '특허', '출원', '자율주행', '차량', '브레이크', '고장', '대응', '시스템', '을', '테크', '사이트', '공개', '현대차그룹', '해당', '기술', '완전', '자율주행', '모빌리티', '전제', '브레이크', '장치', '오류', '발생', '예기치', '상황', '차량', '간', '상호작용', '피해', '최소', '대응', '기술', '설명', '브레이크', '고장', '시스템', '자율주행', '차량', '주행', '도중', '제동', '필요', '상황', '전자', '제어장치', '제동', '장치', '작동', '확인', '반응', '전자', '제어장치', '한번', '보조', '제동', '장치', '제동', '가능', '여부', '판별', '이때', '보조', '제동', '장치', '사용', '불가능', '판단', '전자', '제어장치', '자율주행', '관제시스템', '제동', '시스템', '문제', '신호', '전달', '정보', '전달', '관제시스템', '고장', '차량', '가까이', '주행', '중', '차량', '구조', '신호', '신호', '수', '한', '자율주행', '차량', '고장', '차량', '안전', '정지', '고장', '차량', '관제시스템', '전달한', '정보', '토대', '조향', '시스템', '제', '구조', '진행', '상황', '관제시스템', '전달함', '고장', '차량', '차선', '주행', '접근', '관제시스템', '고장', '차량', '무사', '처리', '주변', '차량들', '해당', '차선', '주행', '지시', '차선', '이동한', '차량', '주행', '속도', '조향', '시스템', '조절', '고장', '차량', '범퍼', '접촉', '차량', '속도', '줄', '한', '차량', '제어', '고장', '차량', '관제시스템', '구조', '완료', '신호', '후', '구조', '작

주제와 연관된 단어들이 자주 출현한 것을 볼 수 있습니다.

In [8]:
# 명사만 추출된 리스트
word_list=temp

# 문서에 출현한 각 명사로부터 Series 객체를 생성 (index는 0부터 시작하는 정수값)
# 0 차량
# 1 고장
# 2 제동
# 3 차량
# 4 고장
# 5 ...
word_list=pd.Series([x for x in word_list if len(x)>1])

# pd.Series.value_counts: Returns a Series containing counts of unique values.
# The resulting object will be in descending order so that the first element is the most frequently-occurring element.  
word_list.value_counts().head(10), data_shuffled.iloc[0]['주제']

(차량       16
 고장       10
 관제시스템     6
 자율주행      6
 제동        6
 시스템       6
 주행        5
 신호        4
 장치        4
 기술        4
 dtype: int64, '모빌리티')

***

##**< DBSCAN 예제 : 기사 분석하기 >**  
기후변화, 유산균, 모빌리티에 관한 기사 제목과 기사 내용을 저장한 데이터에 대해 DBSCAN을 적용합니다.

In [9]:
# 기사 내용중 명사만을 추출하여 docs 리스트에 저장합니다. # 형태소 분석
docs = []
for i in data_shuffled['body']:
    docs.append(hannanum.nouns(i))

# 공백 문자를 넣어서 각 단어를 합칩니다.
for i in range(len(docs)):
    docs[i] = ' '.join(docs[i])

docs[0]

'[자료=현대자동차] 현대차그룹 오늘(26일) 특허 출원 자율주행 차량 브레이크 고장 대응 시스템 을 테크 사이트 공개 현대차그룹 해당 기술 완전 자율주행 모빌리티 전제 브레이크 장치 오류 발생 예기치 상황 차량 간 상호작용 피해 최소 대응 기술 설명 브레이크 고장 시스템 자율주행 차량 주행 도중 제동 필요 상황 전자 제어장치 제동 장치 작동 확인 반응 전자 제어장치 한번 보조 제동 장치 제동 가능 여부 판별 이때 보조 제동 장치 사용 불가능 판단 전자 제어장치 자율주행 관제시스템 제동 시스템 문제 신호 전달 정보 전달 관제시스템 고장 차량 가까이 주행 중 차량 구조 신호 신호 수 한 자율주행 차량 고장 차량 안전 정지 고장 차량 관제시스템 전달한 정보 토대 조향 시스템 제 구조 진행 상황 관제시스템 전달함 고장 차량 차선 주행 접근 관제시스템 고장 차량 무사 처리 주변 차량들 해당 차선 주행 지시 차선 이동한 차량 주행 속도 조향 시스템 조절 고장 차량 범퍼 접촉 차량 속도 줄 한 차량 제어 고장 차량 관제시스템 구조 완료 신호 후 구조 작업 무선 통신 주변 차량 상호작용 사고 완화 것 현대차그룹 브레이크 고장 대응 시스템 기술 특허 미국, 중국, 한국 출원 가능성 희박 만약 상황 전방위 전 체계 구축 자율주행 기술 기초 완성도 말'

처음 한나눔 분석기를 이용해서 명사만 추출한 docs의 구조는 다음과 같은 이중 리스트로 구성되어 있습니다.

- [[문장1], [문장2], ... , [문장70]]

두번째 for문을 이용하면 docs는 다음과 같이 변형되고

- [문장1, 문장2, ... , 문장70]

각 리스트의 원소가 명사들 사이에 공백이 삽입된 텍스트로 바뀝니다.

In [10]:
# TF_IDF로 변환
# sklearn.feature_extraction.text.TfidfVectorizer :
# Converts a collection of raw documents to a matrix of TF-IDF features (document-term matrix)
# ngram_range(min_n, max_n): all values of n such that min_n <= n <= max_n will be used. 
# min_df : ignore terms that have a document frequency lower than the given threshold.
from sklearn.feature_extraction.text import TfidfVectorizer

# n(1 to 5)gram 을 사용하여 위 기사의 명사들을 tfidf vector로 변환
tfidf_vectorizer = TfidfVectorizer(min_df = 3, ngram_range=(1,5))
tfidf_vectorizer.fit(docs)
vector = tfidf_vectorizer.transform(docs).toarray()
print(vector.shape) # num_of_documents X vocab
# DTM (Document, Term Matrix)

(198, 3324)


sklearn.cluster.DBSCAN:
eps : maximum distance between two samples (default 0.5)
min_samples=5 : The number of samples (or total weight) in a neighborhood for a point to be considered as a core point (default 5) 
metric: The metric to use when calculating distance between instances in a feature array. (default 'euclidean')

In [19]:
# sklearn.cluster.DBSCAN: Performs DBSCAN clustering from vector array
# eps: maximum distance between two samples (default 0.5)
# min_samples: The number of samples in a neighborhood for a point to be considered as a core point (default 5) 
# metric: The metric to use when calculating distance between instances in a feature array. (default 'euclidean')
# 'cosine' distance == (1 - cosine similarity)
# ex) If 2 vectors are perfectly the same then the similarity is 1 (angle=0 hence 𝑐𝑜𝑠(𝜃)=1), and the distance is 0 (1–1=0).
from sklearn.cluster import DBSCAN
import numpy as np

vector = np.array(vector)
model = DBSCAN(eps=0.75, min_samples=3, metric = "cosine") 
result = model.fit_predict(vector) # Computes clusters from a data and predict labels.

In [20]:
# 클러스터 번호 -1에는 노이즈 데이터로 판별되어 클러스터링이 안된 문서들이 들어 있음
print('클러스터 번호: 클러스터에 속한 기사의 수')
result_dict = {str(i):list(result).count(i) for i in set(result)} # result의 각 클러스터에 속한 기사수를 카운트
result_dict

클러스터 번호: 클러스터에 속한 기사의 수


{'0': 11,
 '1': 6,
 '2': 6,
 '3': 7,
 '4': 19,
 '5': 5,
 '6': 4,
 '7': 8,
 '8': 6,
 '9': 8,
 '10': 6,
 '11': 3,
 '12': 4,
 '13': 4,
 '14': 3,
 '15': 3,
 '16': 6,
 '17': 3,
 '18': 3,
 '19': 7,
 '20': 3,
 '21': 3,
 '-1': 70}

DBSCAN 을 통해 클러스터링된 index 결과를 데이터 프레임에 추가합니다.

클러스터 -1에는 노이즈 데이터로 판별이 나서 클러스터링이 안된 문서가 들어 있습니다.

In [21]:
# 클러스터 번호를 column으로 추가
data_shuffled['result'] = result

In [22]:
sentences = []
max_cluster_num = 0
for cluster_num in set(result):
    sentence = ''

    # -1은 노이즈 판별이 났거나 클러스터링이 안된 경우
    if(cluster_num == -1): 
        continue
    else:
        max_cluster_num = cluster_num         # 생성된 클러스터의 갯수를 카운트
        temp_df = data_shuffled[data_shuffled['result'] == cluster_num] # cluster num 별로 조회

        print("cluster num : {}".format(cluster_num))
        # zip: 동일한 개수로 이루어진 자료형을 묶어 준다
        for data in zip(temp_df['title'], temp_df['body'], temp_df.iloc): 
            title, body, my_data = data
            print(my_data['주제'], ':', title) # 주제, 기사 제목을 출력
            sentence += body + ' '
        print()
    sentences.append(sentence)

cluster num : 0
모빌리티 : 세븐일레븐, 공유 킥보드 충전·대여 서비스 시작
모빌리티 : 유럽에서도 '카카오T'로 차량 호출…""22개국 확대""
기후위기 : [출발서해안시대] “민주당 최고위원에 호남 인사 전무…호남 소외 우려”
모빌리티 : [ET] ‘교통 취약계층’ 위한 카셰어링 시동…장애인·시니어 일자리까지 창출
모빌리티 : [여의도 사사건건] 양향자 “미중 패권 다툼은 반도체 전쟁, 위기 아닌 기회…우리 기술 패권으로 칩4 주도권 가질 수 있어”
모빌리티 : 아우디 AG, 상반기에 날았다...사상 최대 영업이익
모빌리티 : "버스 예매에 교통카드까지"…롯데카드, '모빌리티 서비스' 개시
모빌리티 : 현대차그룹 팹리스 스타트업 '보스반도체' 투자…미래차 반도체 시너지
기후위기 : [백브리핑] "칠곡서 책 쓰겠다" 이준석, 잠적 아닌 잠적
모빌리티 : [뉴스라이브] 미중 반도체 전쟁 '가속화'...한국에는 위기? 기회?
모빌리티 : 현대차, 상반기 첫 세계 3위…반도체 밀어주기 통했다

cluster num : 1
모빌리티 : 티맵, KB로부터 2천억 투자 유치…이종호 "티맵의 철학은 상생"
모빌리티 : 대리운전업 갈등 2라운드…""대기업 꼼수로 시장 진출""
모빌리티 : 대리운전연합, 최태원 SK 회장에 손편지…"티맵 갈등 중재해달라"
모빌리티 : 대리운전연합, 내일부터 티맵 규탄대회…"사업확장제한 권고 어겨"
모빌리티 : 대리기사들 티맵에 뿔났다…대리운전 비용 높아질까?
모빌리티 : 티맵모빌리티, KB국민은행서 2천억원 유치

cluster num : 2
유산균 : 야쿠르트도 10% 오른다‥hy, 내달 일부 제품 가격 인상
유산균 : 야쿠르트 라이트, 가격 10% 인상…쿠퍼스 2700원 된다
유산균 : hy, 야쿠르트 라이트 등 일부 제품 '가격 인상'
유산균 : "야쿠르트도 오른다"…hy, 내달 일부 제품 가격 인상
유산균 : 야쿠르트도 10% 오른다...hy, 9월부터 일부 제품 인상
유산균 : hy, 9월부터 일부 제품 가격 오른다…"

In [23]:
#클러스터 1번에 속한 데이터 확인
data_cluster1 = data_shuffled[data_shuffled.result==1]
data_cluster1
len(data_cluster1)

6

In [24]:
# num_to_display: 각 클러스터당 출력할 최대 문서 개수
num_to_display = 5
# max_cluster_num: 생성된 클러스터 갯수
print(max_cluster_num + 1)

22


In [25]:
# 각 클러스터 마다 데이터 출력해주기
for i in range(max_cluster_num+1):
  data_cluster = data_shuffled[data_shuffled.result==i]
  data_num = len(data_cluster)

  if len(data_cluster) > num_to_display:
    data_cluster = data_cluster[0:num_to_display]

  print('----- 클러스터 %i -----' %i)
  print('총 데이터 개수: %i' % data_num)
  print(data_cluster)
  print()

----- 클러스터 0 -----
총 데이터 개수: 11
                   date                                              title  \
3   2022-08-22 16:05:00                         세븐일레븐, 공유 킥보드 충전·대여 서비스 시작   
46  2022-08-31 12:26:00                    유럽에서도 '카카오T'로 차량 호출…""22개국 확대""   
54  2022-08-29 10:23:00            [출발서해안시대] “민주당 최고위원에 호남 인사 전무…호남 소외 우려”   
78  2022-08-25 18:11:00         [ET] ‘교통 취약계층’ 위한 카셰어링 시동…장애인·시니어 일자리까지 창출   
94  2022-08-24 16:13:00  [여의도 사사건건] 양향자 “미중 패권 다툼은 반도체 전쟁, 위기 아닌 기회…우리 ...   

                                                 body channel    주제  result  
3   세븐일레븐 공유킥보드 충전 모습. /세븐일레븐 제공 편의점 세븐일레븐이 모빌리티 서...    news  모빌리티       0  
46  카카오모빌리티가 다음 달부터 유럽에서도 카카오T 애플리케이션으로 현지 차량 호출 서...    news  모빌리티       0  
54  [KBS 목포] ■ 인터뷰 자료의 저작권은 KBS에 있습니다. 인용보도 시 출처를 ...    news  기후위기       0  
78  ■ 프로그램명 : 통합뉴스룸ET ■ 코너명 : 호모 이코노미쿠스 ■ 방송시간 : 8...    news  모빌리티       0  
94  양향자 (국민의힘 반도체산업특별위원회 위원장) -반도체는 4차 산업의 기본...외교...    news  모빌리티       0  

----- 클러스터 1 -----
총 데이터 개수: 6